In [1]:
pip install pymongo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.2/677.2 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 21.9 MB/s eta 0:00:00


In [2]:
pip install sqlalchemy

In [3]:
import ast
import pymongo
import re
import sqlalchemy
import json


In [4]:
def convert_python_expression_to_mongo_query(expression):
    # Convert the Python expression to an AST
    ast_tree = ast.parse(expression)

    class MyVisitor(ast.NodeVisitor):
        def __init__(self):
            self.query = None

        def visit_Compare(self, node):
            operator = node.ops[0]

            if isinstance(operator, ast.Eq):
                mongo_op = "$eq"
            elif isinstance(operator, ast.Lt):
                mongo_op = "$lt"
            elif isinstance(operator, ast.Gt):
                mongo_op = "$gt"
            elif isinstance(operator, ast.LtE):
                mongo_op = "$lte"
            elif isinstance(operator, ast.GtE):
                mongo_op = "$gte"
            elif isinstance(operator, ast.NotEq):
                mongo_op = "$ne"
            else:
                raise NotImplementedError(f"Operator {operator} not supported")

            left = self.visit(node.left)
            right = self.visit(node.comparators[0])

            self.query = {
                mongo_op: [left, right],
            }

        def visit_Name(self, node):
            return node.id

        def visit_BinOp(self, node):
            left = self.visit(node.left)
            right = self.visit(node.right)

            if isinstance(node.op, ast.Add):
                return {"$expr": {"$add": [left, right]}}
            elif isinstance(node.op, ast.Sub):
                return {"$expr": {"$subtract": [left, right]}}
            elif isinstance(node.op, ast.Mult):
                return {"$expr": {"$multiply": [left, right]}}
            elif isinstance(node.op, ast.BitOr):  # Handle bitwise OR
                return {"$expr": {"$or": [left, right]}}
            # Handle other binary operators as needed
            else:
                raise NotImplementedError(f"Binary operator {node.op} not supported")

        def visit_BoolOp(self, node):
            op_type = None
            if isinstance(node.op, ast.And):
                op_type = "$and"
            elif isinstance(node.op, ast.Or):
                op_type = "$or"
            elif isinstance(node.op, ast.Not):
                op_type = "$nor"  # Use $nor for negation

            return {op_type: [self.visit(value) for value in node.values]}

        def visit_UnaryOp(self, node):
            if isinstance(node.op, ast.Not):
                operand = self.visit(node.operand)
                return {"$not": operand}
            else:
                raise NotImplementedError(f"Unary operator {node.op} not supported")

        def visit_Call(self, node):
            function_name = node.func.id
            arguments = [self.visit(arg) for arg in node.args]

            if function_name == "all":
                return {arguments[0]: {"$all": arguments[1]}}
            elif function_name == "exists":
                return {arguments[0]: {"$exists": True}}
            elif function_name == "match":
                return {arguments[0]: {"$elemMatch": arguments[1]}}
            elif function_name == "mod":
                return {arguments[0]: {"$mod": arguments[1]}}
            elif function_name == "regex":
                options = arguments[2] if len(arguments) > 2 else None
                return {arguments[0]: {"$regex": arguments[1], "$options": options}}
            # Add support for other functions as needed
            else:
                raise NotImplementedError(f"Function {function_name} not supported")

        def visit_ClassDef(self, node):
            # Handle class definitions
            # Example: class MyCollection(Document):
            class_name = node.name
            # Assuming the class has a meta attribute defining the collection name
            collection_name = node.body[0].value.keywords[0].value.s

            self.query = {
                "class_name": class_name,
                "collection_name": collection_name,
                # Add more details as needed
            }

        def visit_FunctionDef(self, node):
            # Handle function definitions
            # Example: def my_function(arg1, arg2):
            function_name = node.name
            args = [arg.arg for arg in node.args.args]

            self.query = {
                "function_name": function_name,
                "args": args,
                # Add more details as needed
            }


    tree = ast.parse(expression)


    visitor = MyVisitor()


    visitor.visit(tree)


    collection_name = 'Output_Collection'

    # Build the db.collection.validate() command with the generated query
    validate_command = f"db.{collection_name}.validate({visitor.query})"


    print(visitor.query)
    print(validate_command)

In [5]:
def convert_mongo_query_to_sql_query(mongo_query):
    mongo_query_dict = mongo_query


    sql_query = "SELECT * FROM test"

    if isinstance(mongo_query_dict, dict):
        # Initialize a list to store individual conditions
        conditions = []

        for field_name, filter_condition in mongo_query_dict.items():
            for operator, value in filter_condition.items():
                # Convert MongoDB operators to SQL operators
                sql_operator = {
                    '$eq': '=',
                    '$gt': '>',
                    '$lt': '<',
                    '$gte': '>=',
                    '$lte': '<=',
                    '$ne': '!='
                }.get(operator, operator)

                # Build the condition
                conditions.append(f"{field_name} {sql_operator} {value}")

        # Combine conditions using AND in the WHERE clause
        if conditions:
            sql_query += " WHERE " + " AND ".join(conditions)

    return sql_query


In [ ]:
choice = input("Enter 1 to input to convert Python expression to MongoDB Query, or 2 to convert MongoDB Query into SQL Query: ")

if choice == "1":
    expression_to_convert = input("Enter the Python expression to convert: ")
    print("Generating Respective MongoDB Query")
    convert_python_expression_to_mongo_query(expression_to_convert)

elif choice == "2":
    mongo_query = {'name': {'$eq': 'Prudhvi Kari'}}#Hard Coded Value
    print("Generating Respective SQL Query")
    sql_query = convert_mongo_query_to_sql_query(mongo_query)
    print(sql_query)

else:
    print("Invalid choice. Please enter 1 or 2.")

EXAMPLE PYTHON EXPRESSION :- x = 10 + y and z > 5 or not a == 1 or b == 2

EXAMPLE MONGO QUERY :- {'name': {'$eq': 'Prudhvi Kari'}}

**BETA 1**

Status : COMPLETED

More Info:- Now Supports Arthimatic Operators,Bitwise Operators,Comparison Operators

In [ ]:
def convert_mongo_query_to_sql_query(mongo_query):
    # Convert the MongoDB query to a dictionary
    mongo_query_dict = mongo_query

    sql_query = "SELECT * FROM test"

    if isinstance(mongo_query_dict, dict):
        # Initialize a list to store individual conditions
        conditions = []

        for field_name, filter_condition in mongo_query_dict.items():
            for operator, value in filter_condition.items():
                # Convert MongoDB operators to SQL operators
                sql_operator = {
                    '$eq': '=',
                    '$gt': '>',
                    '$lt': '<',
                    '$gte': '>=',
                    '$lte': '<=',
                    '$ne': '!=',
                    '$add': '+',
                    '$subtract': '-',
                    '$multiply': '*',
                    '$divide': '/',
                    '$mod': '%',
                    '$bitwiseAnd': '&',
                    '$bitwiseOr': '|',
                    '$bitwiseXor': '^'
                }.get(operator, operator)

                # Build the condition
                if operator in ('$add', '$subtract', '$multiply', '$divide', '$mod'):
                    conditions.append(f"{field_name} {sql_operator} {value}")
                elif operator in ('$bitwiseAnd', '$bitwiseOr', '$bitwiseXor'):
                    conditions.append(f"{field_name} {sql_operator} {value}")
                else:
                    conditions.append(f"{field_name} {sql_operator} '{value}'")

        # Combine conditions using AND in the WHERE clause
        if conditions:
            sql_query += " WHERE " + " AND ".join(conditions)

    return sql_query

# Example usage
mongo_query = {'age': {'$gt': 25}, 'salary': {'$bitwiseAnd': 15}}
sql_query = convert_mongo_query_to_sql_query(mongo_query)
print(sql_query)


**BETA 1.1**


More Info:- Support for Compound Operators & Logical Operators

In [ ]:
import sqlalchemy

def convert_mongo_query_to_sql_query(mongo_query):
    sql_query = "SELECT * FROM test"

    if isinstance(mongo_query, dict):
        # Initialize a list to store individual conditions
        conditions = []

        for field_name, filter_condition in mongo_query.items():
            if field_name == "$and" or field_name == "$or":
                # Handle logical operators with recursive calls
                sub_conditions = convert_mongo_query_to_sql_query(filter_condition)
                conditions.append(f"({sub_conditions})")
            else:
                for operator, value in filter_condition.items():
                    # Convert MongoDB operators to SQL operators
                    sql_operator = {
                        '$eq': '=',
                        '$gt': '>',
                        '$lt': '<',
                        '$gte': '>=',
                        '$lte': '<=',
                        '$ne': '!=',
                    }.get(operator, operator)

                    # Build the condition
                    conditions.append(f"{field_name} {sql_operator} '{value}'")

        # Combine conditions using AND in the WHERE clause
        if conditions:
            sql_query += " WHERE " + " AND ".join(conditions)

    return sql_query

# Example usage
mongo_query = {
    '$and': [
        {'age': {'$gte': 25}},
        {'$or': [
            {'salary': {'$gt': 50000}},
            {'position': {'$eq': 'Manager'}}
        ]}
    ]
}
mongo_quey = " "
sql_query = convert_mongo_query_to_sql_query(mongo_query)
print(sql_query)


TESTING THE PQL TOOL FOR Find Queries

Schema-Free Example
Schema-Aware Example
Referencing Fields
Operators
Boolean Logic
Functions
Geo Queries
Aggregation Queries
Referencing Fields
Arithmetic Operators
Logical Operators
Date Operators
String Operators





In [ ]:
import ast

def convert_python_expression_to_mongo_query(expression):
    # Convert the Python expression to an AST
    ast_tree = ast.parse(expression)

    # Create a visitor class to convert the AST to a MongoDB query
    class MyVisitor(ast.NodeVisitor):
        def __init__(self):
            self.query = None

        def visit_Compare(self, node):
            operator = node.ops[0]

            if isinstance(operator, ast.Eq):
                mongo_op = "$eq"
            elif isinstance(operator, ast.Lt):
                mongo_op = "$lt"
            elif isinstance(operator, ast.Gt):
                mongo_op = "$gt"
            elif isinstance(operator, ast.LtE):
                mongo_op = "$lte"
            elif isinstance(operator, ast.GtE):
                mongo_op = "$gte"
            elif isinstance(operator, ast.NotEq):
                mongo_op = "$ne"
            else:
                raise NotImplementedError(f"Operator {operator} not supported")

            left = self.visit(node.left)
            right = self.visit(node.comparators[0])

            self.query = {
                mongo_op: [left, right],
            }

        def visit_Name(self, node):
            return node.id

        def visit_BinOp(self, node):
            left = self.visit(node.left)
            right = self.visit(node.right)

            if isinstance(node.op, ast.Add):
                return {"$expr": {"$add": [left, right]}}
            elif isinstance(node.op, ast.Sub):
                return {"$expr": {"$subtract": [left, right]}}
            elif isinstance(node.op, ast.Mult):
                return {"$expr": {"$multiply": [left, right]}}
            elif isinstance(node.op, ast.BitOr):  # Handle bitwise OR
                return {"$expr": {"$or": [left, right]}}
            # Handle other binary operators as needed
            else:
                raise NotImplementedError(f"Binary operator {node.op} not supported")

        def visit_BoolOp(self, node):
            op_type = None
            if isinstance(node.op, ast.And):
                op_type = "$and"
            elif isinstance(node.op, ast.Or):
                op_type = "$or"
            elif isinstance(node.op, ast.Not):
                op_type = "$nor"  # Use $nor for negation

            return {op_type: [self.visit(value) for value in node.values]}

        def visit_UnaryOp(self, node):
            if isinstance(node.op, ast.Not):
                operand = self.visit(node.operand)
                return {"$not": operand}
            else:
                raise NotImplementedError(f"Unary operator {node.op} not supported")

        def visit_Call(self, node):
            function_name = node.func.id
            arguments = [self.visit(arg) for arg in node.args]

            if function_name == "all":
                return {arguments[0]: {"$all": arguments[1]}}
            elif function_name == "exists":
                return {arguments[0]: {"$exists": True}}
            elif function_name == "match":
                return {arguments[0]: {"$elemMatch": arguments[1]}}
            elif function_name == "mod":
                return {arguments[0]: {"$mod": arguments[1]}}
            elif function_name == "regex":
                options = arguments[2] if len(arguments) > 2 else None
                return {arguments[0]: {"$regex": arguments[1], "$options": options}}
            # Add support for other functions as needed
            else:
                raise NotImplementedError(f"Function {function_name} not supported")

        def visit_Expr(self, node):
            self.query = self.visit(node.value)

    # Create the AST from the provided expression
    tree = ast.parse(expression)

    # Create an instance of the custom visitor
    visitor = MyVisitor()

    # Traverse the abstract syntax tree with the visitor
    visitor.visit(tree)

    # Assuming you have a MongoDB collection named 'your_collection'
    collection_name = 'your_collection'

    # Build the db.collection.validate() command with the generated query
    validate_command = f"db.{collection_name}.validate({visitor.query})"

    # Print the resulting query and validate command
    print(visitor.query)
    print(validate_command)

# Example usage
expression_to_convert = "x = 10 + y and z > 5 or not a == 1 or b == 2"
convert_python_expression_to_mongo_query(expression_to_convert)


**BETA 1.2:**

STATUS : IN PROGRESS

REQUIREMENT 1: Need support for Joins

python_expression = "(user.age > 25) & (user.gender == 'male') | (order.city == 'New York')"

APPROACH 1:

context = {'user': 'users', 'order': 'orders'}

Need to provide a dictionary (context) that maps the variable names in your expression to the corresponding MongoDB collections or fields. The code doesn't handle full-fledged joins as MongoDB doesn't directly support them. It assumes that you have denormalized data or a structure where you can directly reference fields from related documents.


REQUIREMENT 2: Declarative schema generation syntax

schema = MongoSchema()
schema.add_field('user', 'user')
schema.add_field('order', 'order')

MongoSchema class to define the expected fields and their types. The add_field method allows you to add fields to the schema, and the validate method checks if the provided context adheres to the schema.